In [1]:
import os
import numpy as np
import scipy.sparse as sp
import torch
import torch.nn as nn
import torch.nn.functional as F
import sys
import math
import pywt

In [ ]:
def multimodal_multiscale_wavelet_align(image_embedding, text_embedding):
    '''
    Desc: 多模态多尺度小波变换对齐
    Args:
        image_embedding: 图像embedding, image_embedding.shape: torch.Size([7050, 64]) 
        text_embedding: 文本embedding, text_embedding.shape: torch.Size([7050, 64]), 7050个物品,每个物品编码为64维向量
    Function:
        对图像和文本嵌入进行多模态多尺度小波变换对齐 ：
        Steps1: 选取模态领域适应的合适小波基['harr', 'db1-20', 'bior1.3'], 选择高通滤波器的下采样分级尺度level,本文默认选择level3, 注意输入的特征向量是一维度，因此暂时不考虑水平、垂直、对角。
        Steps2: Multimodal Wavelet Transform Learning Project： 获取图像模态特征的image_coeffs（一个低通分量，三个高通分量）；文本模态特征的text_coeffs（一个低通分量，三个高通分量）
        Steps3: 多模态多尺度频域空间对齐 Multimodal Multi-Scale Frequency Domain Align：
                3.1低频信号对齐
                    1.对于低频信号（第 3 级水平低频分量），首先对图像模态和文本模态下的低频信号分别进行LoRA(奇异值分解（SVD）),用于去除冗余信息，保留主要特征。 
                    2.然后,再分别对进行归一化，对图像低频信号，可将其像素值归一化到特定范围，如 [0, 1] 或 [-1, 1]，以消除不同图像之间的亮度差异等影响。 对于文本低频信号，可对词向量或特征向量进行归一化，使不同文本的特征具有可比性，例如采用 L2 归一化。
                    3.然后再将LoRa和归一化后的两个模态的特征向量进行对齐，对齐方式采用语义注意力机制， 生成一个新的融合模态第 3 级水平低频分量
                3.2 高频信号多尺度对齐 TODO：高频信号不同等级能量是否才用不同对齐方式？
                    1.对图像和文本的第 3 级水平高频分量进行对齐，对齐方式采用频域能量对齐，生成一个新的融合模态第 3 级水平高频分量
                    2.对图像和文本的第 2 级水平高频分量进行对齐， 生成一个新的融合模态第 2 级水平高频分量
                    3.对图像和文本的第 1 级水平高频分量分别先进行去噪，再进行对齐， 生成一个新的融合模态第 1 级水平高频分量
        Steps4： Multimodal Wavelet Inverse Transform 多模态小波重建

                对融合模态的低频分量和3个高频分量进行小波逆变换，重建为一个多模态频域对齐以及语义域融合的特征向量fusion_wave
                对图像模态、文本模态经过上述变换后，进行小波逆变换，重建为新的图像image_embedding_wave、文本模态特征向量text_embedding_wave。
    Returns:
        image_embedding_wave.shape: torch.Size([7050, 64]) text_embedding_wave.shape: torch.Size([7050, 64]) fusion_wave.shape: torch.Size([7050, 64])
    '''
    device = image_embedding.device
    print("device:", device)
    # print("image_embedding:", image_embedding.shape)
    # 转换为numpy数组进行小波变换
    # image_np = image_embedding.detach().cpu().numpy()
    # text_np = text_embedding.detach().cpu().numpy()

    # 定义小波类型
    wavelet = 'db4'
    level = 3 

    # 对图像和文本嵌入进行小波变换
    '''
    原始信号 如果输入是 (256, 256) 的图像，axis=1（水平方向分解），level=3：
│
    ├── cA3（最低频，最模糊的近似）      主体信号，轮廓 （保留）  (256, 32)  第 3 级水平低频分量
    │
    ├── cD3（第3级细节，较大尺度的高频） 大尺度边缘 最精细的细节信息 (256, 32) 第 3 级水平高频分量
    │
    ├── cD2（第2级细节，中等尺度的高频） 中尺度细节，更细微的边缘和纹理变化 (256, 64) 第 2 级水平高频分量
    │
    └── cD1（第1级细节，最细尺度的高频） 高频噪声->去噪 (256, 128) 第 1 级水平高频分量
    '''
    image_coeffs = wavelet_decompose(image_embedding, wavelet, level=level, axis=1, device=device)
    text_coeffs = wavelet_decompose(text_embedding, wavelet, level=level, axis=1, device=device)

    '''
    image_coeffs shapes:
        wavelet_decompose Level 4: (7050, 14)
        wavelet_decompose Level 3: (7050, 14)
        wavelet_decompose Level 2: (7050, 21)
        wavelet_decompose Level 1: (7050, 35)
    text_coeffs shapes:
        wavelet_decompose Level 4: (7050, 14)
        wavelet_decompose Level 3: (7050, 14)
        wavelet_decompose Level 2: (7050, 21)
        wavelet_decompose Level 1: (7050, 35)
    '''
    # len(image_coeffs): 4 len(text_coeffs): 4
    # print("len(image_coeffs):", len(image_coeffs), "len(text_coeffs):", len(text_coeffs))
    # 对每一级系数进行对齐和融合
    # ===================== 多模态多尺度频域对齐 =====================
    fused_coeffs = []
    img_coeffs_proc = []
    txt_coeffs_proc = []

    for i, (img_coeff, txt_coeff) in enumerate(zip(image_coeffs, text_coeffs)):        
        # 低频分量
        if i == 0: 
            img_proc = process_low(img_coeff, modality='image')
            txt_proc = process_low(txt_coeff, modality='text')
            # 融合
            fused = fuse_low(img_proc, txt_proc)
        # 高频分量
        else:       
            level_type = len(image_coeffs)-i  # 计算当前层级(3,2,1)
            # 直接传递原始系数用于重建
            img_proc = img_coeff  
            txt_proc = txt_coeff
            # 融合处理
            fused = fuse_high(img_coeff, txt_coeff, level_type)
        # 保存处理结果
        img_coeffs_proc.append(img_proc)
        txt_coeffs_proc.append(txt_proc)
        fused_coeffs.append(fused)


    # 进行小波逆变换
    image_embedding_wave = wavelet_reconstruct(img_coeffs_proc, wavelet, axis=1, device=device)
    text_embedding_wave = wavelet_reconstruct(txt_coeffs_proc, wavelet, axis=1, device=device)
    fusion_wave = wavelet_reconstruct(fused_coeffs, wavelet, axis=1, device=device)

    return image_embedding_wave, text_embedding_wave, fusion_wave

def wavelet_decompose(x, wavelet='db4', level=3, axis=1, device='cuda'):
    '''
        小波变换分解
        Steps1: 选取模态领域适应的合适小波基['harr', 'db1-20', 'bior1.3'], 选择高通滤波器的下采样分级尺度level,本文默认选择level3, 注意输入的特征向量是一维度，因此暂时不考虑水平、垂直、对角。

    '''
    x_np = x.detach().cpu().numpy()
    coeffs = pywt.wavedec(x_np, wavelet, level=level, axis=axis)
    return [torch.tensor(c, device=device, dtype=torch.float32) for c in coeffs]
    # return pywt.wavedec(x, wavelet, level=level, axis=axis)

    # ===================== 小波重建 =====================
def wavelet_reconstruct(coeffs, wavelet='db4', axis=1, device='cuda'):
    coeffs_np = [c.detach().cpu().numpy() for c in coeffs]
    rec = pywt.waverec(coeffs_np, wavelet, axis=1)
    return torch.tensor(rec, device=device, dtype=torch.float32)

def process_low(coeff, modality='image'):
    """
        低频信号: SVD降维+归一化
    
    """
    # SVD降维
    U, S, V = torch.svd_lowrank(coeff, q=min(coeff.shape)//2)
    recon = U @ torch.diag(S) @ V.T

    # 归一化
    if modality == 'image':
        recon = 2 * (recon - recon.min())/(recon.max() - recon.min() + 1e-8) - 1
    else:
        recon = torch.nn.functional.normalize(recon, p=2, dim=1)
    return recon


def fuse_low(img_low, txt_low):
    """
        低频信号对齐: 基于语义的attention
    """
    # 语义注意力机制
    attn = torch.stack([img_low, txt_low], dim=1)  # [N, 2, D]
    attn_weights = torch.softmax(attn @ attn.transpose(1, 2), dim=1)  # [N, 2, 2]
    # 加权融合
    w_img = attn_weights[:, 0, 0].unsqueeze(1)
    w_txt = attn_weights[:, 0, 1].unsqueeze(1)
    return w_img * img_low + w_txt * txt_low


def fuse_high(img_high, txt_high, level_type):
    """
        低频信号对齐: 基于语义的attention
    """
    eps = 1e-8  # 防止除以零
    
    if level_type == 3:  # 能量对齐
        energy_img = torch.norm(img_high, dim=1, keepdim=True)
        energy_txt = torch.norm(txt_high, dim=1, keepdim=True)
        return (energy_img*img_high + energy_txt*txt_high)/(energy_img + energy_txt + eps)
        
    elif level_type == 2:  # 直接平均
        return (img_high + txt_high) / 2
        
    elif level_type == 1:  # 去噪后融合
        def denoise(x):
            threshold = torch.median(torch.abs(x)) / 0.6745
            return torch.sign(x) * torch.relu(torch.abs(x) - threshold)
            
        return (denoise(img_high) + denoise(txt_high)) / 2





In [46]:
img = torch.randn(7050, 64).cuda()
txt = torch.randn(7050, 64).cuda()



In [47]:
img_wave, txt_wave, fusion = multimodal_multiscale_wavelet_align(img, txt)

print("图像重建特征形状:", img_wave.shape)    # [7050,64]
print("文本重建特征形状:", txt_wave.shape)    # [7050,64]
print("融合特征形状:", fusion.shape)        # [7050,64]

device: cuda:0
图像重建特征形状: torch.Size([7050, 64])
文本重建特征形状: torch.Size([7050, 64])
融合特征形状: torch.Size([7050, 64])


In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from pytorch_wavelets import DWT1DForward, DWT1DInverse

class MultiModalWaveletInterestAttention(nn.Module):
    '''
    多模态小波兴趣注意力机制（门控融合+对比学习）
    '''
    def __init__(self, 
                 embed_dim, 
                 wavelet_name='db4', 
                 contrastive_temperature=0.5):
        super().__init__()
        self.embed_dim = embed_dim
        self.contrastive_temperature = contrastive_temperature
        
        # ------------------- 小波变换 -------------------
        self.dwt = DWT1DForward(wave=wavelet_name, J=1)  # 固定1级分解（低频+高频）
        self.idwt = DWT1DInverse(wave=wavelet_name)
        
        # ------------------- 门控融合组件 -------------------
        # 低频门控（融合content/side/fusion三模态低频）
        self.low_gate = nn.Sequential(
            nn.Linear(3 * (embed_dim//2), 3),  # 输入3个低频特征（每个dim=embed_dim//2）
            nn.Softmax(dim=-1)
        )
        # 高频门控（融合三模态高频）
        self.high_gate = nn.Sequential(
            nn.Linear(3 * (embed_dim//2), 3),
            nn.Softmax(dim=-1)
        )
        
        # ------------------- 频域投影 -------------------
        self.low_proj = nn.Linear(embed_dim//2, embed_dim)  # 低频恢复原始维度
        self.high_proj = nn.Linear(embed_dim//2, embed_dim)  # 高频恢复原始维度
        
        # ------------------- 动态注意力 -------------------
        self.attention = nn.Sequential(
            nn.Linear(2 * embed_dim, embed_dim//4),  # 输入：低频+高频投影后的特征
            nn.LayerNorm(embed_dim//4),
            nn.GELU(),
            nn.Linear(embed_dim//4, 2),  # 输出低频和高频权重
            nn.Softmax(dim=-1)
        )
        
        # ------------------- 对比学习头 -------------------
        self.low_contrast = nn.Sequential(
            nn.Linear(embed_dim//2, embed_dim//2),
            nn.LayerNorm(embed_dim//2),
            nn.ReLU()
        )
        self.high_contrast = nn.Sequential(
            nn.Linear(embed_dim//2, embed_dim//2),
            nn.LayerNorm(embed_dim//2),
            nn.ReLU()
        )
        
        # ------------------- 残差连接 -------------------
        self.res_norm = nn.LayerNorm(embed_dim)

    def wavelet_decomp(self, x):
        """单级小波分解：返回低频和高频"""
        x = x.unsqueeze(1)  # [B,1,D]
        cA, cD = self.dwt(x)
        return cA.squeeze(1), cD[0].squeeze(1)  # [B,D/2], [B,D/2]

    def gated_fusion(self, inputs, gate):
        """通用门控融合函数"""
        # inputs shape: [B, 3, D/2]（3个模态：content/side/fusion）
        weights = gate(inputs.reshape(-1, 3 * (self.embed_dim//2)))  # [B*3, 3] → [B,3,3]
        weights = weights.softmax(dim=-1).unsqueeze(-1)  # [B,3,3,1]
        fused = torch.sum(inputs * weights, dim=1)  # 加权求和 → [B, D/2]
        return fused

    def forward(self, content_embeds, side_embeds, labels=None):
        B = content_embeds.shape[0]
        fusion_embeds = content_embeds + side_embeds  # 基础融合特征
        
        # ------------------- 1. 小波分解 -------------------
        # 分解三模态特征
        c_low, c_high = self.wavelet_decomp(content_embeds)
        s_low, s_high = self.wavelet_decomp(side_embeds)
        f_low, f_high = self.wavelet_decomp(fusion_embeds)
        
        # ------------------- 2. 门控融合低频和高频 -------------------
        # 拼接三模态特征为 [B,3,D/2]
        low_input = torch.stack([c_low, s_low, f_low], dim=1)  # [B,3,D/2]
        print("low_input.shape:", low_input.shape)
        high_input = torch.stack([c_high, s_high, f_high], dim=1)  # [B,3,D/2]
        
        # 门控融合
        low_fused = self.gated_fusion(low_input, self.low_gate)  # [B,D/2]
        high_fused = self.gated_fusion(high_input, self.high_gate)  # [B,D/2]
        
        # ------------------- 3. 频域特征投影回原始维度 -------------------
        low_proj = self.low_proj(low_fused)  # [B,D/2] → [B,D]
        high_proj = self.high_proj(high_fused)  # [B,D/2] → [B,D]
        
        # ------------------- 4. 动态频域注意力 -------------------
        attn_input = torch.cat([low_proj, high_proj], dim=-1)  # [B, 2D]
        weights = self.attention(attn_input)  # [B,2]：[low_weight, high_weight]
        final_embedding = weights[:,0:1] * low_proj + weights[:,1:2] * high_proj
        
        # ------------------- 5. 残差连接 -------------------
        residual = self.res_norm(fusion_embeds)  # 归一化原始融合特征
        output = final_embedding + residual  # 残差连接
        
        # ------------------- 6. 对比学习（可选） -------------------
        contrastive_loss = None
        if labels is not None and self.training:
            contrastive_loss = self.compute_contrastive_loss(
                low_fused, high_fused, labels
            )
        
        return output, low_fused, high_fused, contrastive_loss

    def compute_contrastive_loss(self, low_fused, high_fused, labels):
        """计算低频（热门）和高频（小众）的对比损失"""
        # 低频对比：同类样本低频应相似
        low_repr = F.normalize(self.low_contrast(low_fused), dim=-1)  # [B, D/2]
        low_sim = torch.matmul(low_repr, low_repr.T) / self.contrastive_temperature
        
        # 高频对比：同类样本高频应差异大
        high_repr = F.normalize(self.high_contrast(high_fused), dim=-1)  # [B, D/2]
        high_sim = torch.matmul(high_repr, high_repr.T) / self.contrastive_temperature
        
        # 正负样本掩码
        pos_mask = (labels.unsqueeze(1) == labels.unsqueeze(0)).float()
        neg_mask = 1 - pos_mask
        
        # 低频损失：最大化正相似性
        low_loss = -torch.log(
            (torch.exp(low_sim) * pos_mask).sum(dim=-1) / 
            (torch.exp(low_sim).sum(dim=-1) + 1e-8)
        ).mean()
        
        # 高频损失：最小化正相似性
        high_loss = torch.log(
            (torch.exp(-high_sim) * pos_mask).sum(dim=-1) / 
            (torch.exp(-high_sim).sum(dim=-1) + 1e-8)
        ).mean()
        
        return low_loss + high_loss

In [11]:
import torch
import torch.nn as nn

class MultiModalWaveletInterestAttention(nn.Module):
    '''
        Desc: 多模态小波兴趣注意力机制（优化版）
            必要性: 当前捕获用户兴趣偏好的方法无法分离用户对物品的Hot兴趣和小众兴趣, 同时融合兴趣的方式是content_embeds + side_embeds
            -------------------------------------------
            核心改进：
                1. 门控融合机制：分别对低频/高频特征设计门控，动态调节content与side特征的贡献比例
                2. 注意力显式加权：直接对分离后的低频/高频特征加权，物理意义更明确（低频=热门兴趣，高频=小众兴趣）
                3. 对比学习支持：显式输出分离后的低频/高频兴趣特征，便于后续对比损失计算
            -------------------------------------------
            基于小波变换的多模态兴趣偏好感知, 核心流程：
            1. 兴趣分离：对content_embeds、side_embeds、融合特征分别小波分解，得到低频(公共兴趣)和高频(个性化兴趣)分量
            2. 门控融合：在低频域用门控加权融合多源特征，高频域用门控增强个性化特征
            3. 注意力加权：对融合后的低频/高频特征显式加权，平衡热门与小众兴趣
            4. 多尺度重构：通过小波逆变换得到多尺度兴趣感知特征，并保留分离的低/高频兴趣用于对比学习
        Args:
            content_embeds.shape: torch.Size([B, D])  用户和物品的embedding（B: batch_size, D: embed_dim）
            side_embeds.shape: torch.Size([B, D])    多模态融合的embedding
        Returns:
            mm_interest_prefer_aware_embeds: torch.Size([B, D])  多尺度兴趣感知embedding
            low_freq_interest: torch.Size([B, D])  分离后的低频（热门）兴趣特征
            high_freq_interest: torch.Size([B, D])  分离后的高频（小众）兴趣特征
    '''
    def __init__(self, embed_dim, wavelet_name='db4', decomp_level=1):
        super().__init__()
        self.embed_dim = embed_dim
        self.wavelet_name = wavelet_name
        
        # 小波变换与逆变换（支持自动梯度计算）
        self.dwt = DWT1DForward(wave=wavelet_name, J=decomp_level)  # 需确保DWT1DForward已正确实现
        self.idwt = DWT1DInverse(wave=wavelet_name)                  # 需确保DWT1DInverse已正确实现
        
        # 改进1：低频/高频门控融合层（动态调节content与side的贡献比例）
        self.low_gate = nn.Sequential(
            nn.Linear(embed_dim, 1),  # 输出低频融合权重（0-1之间）
            nn.Sigmoid()
        )
        self.high_gate = nn.Sequential(
            nn.Linear(embed_dim, 1),  # 输出高频融合权重（0-1之间）
            nn.Sigmoid()
        )
        
        # 改进2：显式注意力机制（直接对低频/高频特征加权）
        self.attention = nn.Sequential(
            nn.Linear(embed_dim, embed_dim//4),  # 降低维度减少计算量
            nn.LayerNorm(embed_dim//4),
            nn.GELU(),
            nn.Linear(embed_dim//4, 2),          # 输出2个权重（低频alpha和高频beta）
            nn.Softmax(dim=-1)                   # 确保权重和为1，平衡两种兴趣
        )
        
        # 低频/高频投影层（保持维度一致性）
        self.low_proj = nn.Sequential(
            nn.Linear(embed_dim//2, embed_dim//2),
            nn.LeakyReLU()
        )
        self.high_proj = nn.Sequential(
            nn.Linear(embed_dim//2, embed_dim//2),
            nn.BatchNorm1d(embed_dim//2),
            nn.LeakyReLU()
        )
        
        # 残差归一化
        self.norm = nn.LayerNorm(embed_dim)

    def wavelet_decomp(self, x):
        """可微分小波分解（输出低频cA和高频cD分量）"""
        x = x.unsqueeze(1)  # [B, 1, D]  增加通道维度（适应小波变换输入要求）
        cA, cD = self.dwt(x)  # cA: [B, 1, D/2], cD: list([B, 1, D/2])（层数=decomp_level）
        return cA.squeeze(1), cD[0].squeeze(1)  # 移除通道维度 -> [B, D/2]

    def wavelet_recon(self, cA, cD):
        """可微分小波重构（从低频cA和高频cD恢复原始维度）"""
        cA = cA.unsqueeze(1)  # [B, 1, D/2] 恢复通道维度
        cD = [cD.unsqueeze(1)]  # 包装为列表（适应逆变换输入要求）
        return self.idwt((cA, cD)).squeeze(1)  # 重构后移除通道维度 -> [B, D]

    def forward(self, content_embeds, side_embeds):
        '''
        content_embeds.shape: torch.Size([B, 64])  用户和物品的embedding
        side_embeds.shape: torch.Size([B, 64])    多模态融合的embedding
        '''
        # ------------------------ 步骤1：多源特征小波分解 ------------------------
        # 分解content特征（用户-物品交互信息）
        content_cA, content_cD = self.wavelet_decomp(content_embeds)  # [B, 32], [B, 32]
        # 分解side特征（多模态辅助信息）
        side_cA, side_cD = self.wavelet_decomp(side_embeds)          # [B, 32], [B, 32]
        # 分解原始融合特征（content + side的初始融合）
        fusion_embeds = content_embeds + side_embeds
        fusion_cA, fusion_cD = self.wavelet_decomp(fusion_embeds)    # [B, 32], [B, 32]

        # ------------------------ 步骤2：门控融合低频/高频特征 ------------------------
        # 低频（热门兴趣）融合：门控加权content_cA、side_cA、fusion_cA
        # 门控输入：原始融合特征（包含全局信息）
        low_gate_weight = self.low_gate(fusion_embeds)  # [B, 1]  权重范围[0,1]
        low_fused = (
            low_gate_weight * (content_cA + side_cA) +  # 门控加权content与side的低频分量
            (1 - low_gate_weight) * fusion_cA            # 保留原始融合低频分量
        )
        low_fused = self.low_proj(low_fused)  # 投影层增强特征表达

        # 高频（小众兴趣）融合：门控增强个性化特征（content_cD与side_cD的逐元素乘积+融合高频）
        high_gate_weight = self.high_gate(fusion_embeds)  # [B, 1]  权重范围[0,1]
        high_fused = (
            high_gate_weight * (content_cD * side_cD) +   # 逐元素乘积增强个性化（仅保留共同高频特征）
            (1 - high_gate_weight) * fusion_cD            # 保留原始融合高频分量
        )
        high_fused = self.high_proj(high_fused)  # 投影层增强特征表达

        # ------------------------ 步骤3：注意力显式加权低频/高频 ------------------------
        # 拼接低频/高频特征作为注意力输入（显式关联两种兴趣）
        attn_input = torch.cat([low_fused, high_fused], dim=-1)  # [B, 64]
        alpha, beta = self.attention(attn_input).chunk(2, dim=-1)  # [B, 1], [B, 1]  注意力权重

        # 加权融合并重构多尺度兴趣特征
        weighted_low = alpha * low_fused  # 注意力加权后的低频兴趣
        weighted_high = beta * high_fused  # 注意力加权后的高频兴趣
        mm_interest_prefer_aware_embeds = self.wavelet_recon(weighted_low, weighted_high)  # [B, 64]

        # ------------------------ 步骤4：输出分离的低/高频兴趣（用于对比学习） ------------------------
        # 低频兴趣：仅用content与side的低频分量重构（纯公共兴趣）
        low_freq_interest = self.wavelet_recon(content_cA, side_cA)  # [B, 64]
        # 高频兴趣：仅用content的高频分量与融合高频分量重构（纯个性化兴趣）
        high_freq_interest = self.wavelet_recon(content_cD, fusion_cD)  # [B, 64]

        # 残差连接+归一化（增强训练稳定性）
        mm_interest_prefer_aware_embeds = self.norm(mm_interest_prefer_aware_embeds + fusion_embeds)

        return mm_interest_prefer_aware_embeds, low_freq_interest, high_freq_interest


# 新增兴趣Switcher

In [ ]:
class MultiModalWaveletInterestAttention(nn.Module):
    '''
        Desc: 多模态小波兴趣注意力机制
            必要性:当前捕获用户兴趣偏好的方法无法分离用户对物品的Hot兴趣和小众兴趣,  同时融合兴趣的方式是content_embeds + side_embeds
            -------------------------------------------
            优点：
                小波变换能有效分离频域特征，适合兴趣分解
                低频/高频的区分符合热门/小众兴趣的特性
                Attention加权可以动态平衡两种兴趣
            -------------------------------------------
            基于小波变换的，多模态兴趣偏好感知, 初步思路是：
            1. 兴趣分离:分别对content_embeds和fusion_embeds以及融合兴趣(content_embeds + side_embeds)进行小波分解，分离出在低频(对多模态公共兴趣，热门兴趣)和高频特征(个性化兴趣，小众兴趣)中进行用户兴趣。
            2. 分别在低频和高频域进行content_embeds和 side_embeds进行兴趣偏好融合，目前使用的是element-wise add,用简单高效优雅的方式(还有哪些方式)。
            然后，将融合后的低频特征(热门兴趣) 和 高频特征(小众兴趣) 设计一个attention进行加权
            3. 多尺度兴趣感知:最后再小波逆变换，得到多尺度用户兴趣感知embeeding 以及高频兴趣和低频兴趣(用于对比学习)
        Args:
            content_embeds.shape: torch.Size([26495, 64])  用户和物品的embedding
            side_embeds.shape: torch.Size([26495, 64])    多模态融合的embeeding
        Returns:
            mm_interest_prefer_aware_embeds: torch.Size([26495, 64]) ，低频注意力兴趣 torch.Size([26495, 64]) ，高频注意力兴趣 torch.Size([26495, 64]) 
    '''
    def __init__(self, embed_dim, wavelet_name='db1', decomp_level=1):
        super().__init__()
        self.embed_dim = embed_dim
        self.wavelet_name = wavelet_name
        
        # 小波变换与逆变换（支持自动梯度计算）
        self.dwt = DWT1DForward(wave=wavelet_name, J=decomp_level)
        self.idwt = DWT1DInverse(wave=wavelet_name)
        
        # 增强的注意力机制
        self.attention = nn.Sequential(
            nn.Linear(embed_dim, embed_dim//8),
            nn.GELU(),
            nn.Softmax(dim=-1)
        )
        
        # 低频融合投影层
        self.low_fusion = nn.Sequential(
            nn.Linear(embed_dim//2 , embed_dim//2),
            nn.LeakyReLU(),
        )
        
        # 高频融合投影层
        self.high_fusion = nn.Sequential(
            nn.Linear(embed_dim//2 , embed_dim//2),
            nn.BatchNorm1d(embed_dim//2),
            nn.LeakyReLU(),
        )
        
        # 残差归一化
        self.norm = nn.LayerNorm(embed_dim)

        # 新增：低频门控（调节content/side/fusion的低频贡献）
        self.low_gate = nn.Sequential(
            nn.Linear( embed_dim//2 * 3 , 3),  # 3个输入源（content/side/fusion）的低频权重
            nn.Softmax(dim=-1)
        )
        # 新增：高频门控（调节content/side/fusion的高频贡献）
        self.high_gate = nn.Sequential(
            nn.Linear( embed_dim//2 * 3, 3),  # 3个输入源的高频权重
            nn.Softmax(dim=-1)
        )
    def wavelet_decomp(self, x):
        """可微分小波分解"""
        x = x.unsqueeze(1)  # [B,1,D]
        cA, cD = self.dwt(x)
        return cA.squeeze(1), cD[0].squeeze(1)
    
    def wavelet_recon(self, cA, cD):
        """可微分小波重构"""
        cA = cA.unsqueeze(1)
        cD = [cD.unsqueeze(1)]
        return self.idwt((cA, cD)).squeeze(1)
    
    def forward(self, content_embeds, side_embeds):
        '''
        content_embeds.shape: torch.Size([26495, 64])  用户和物品的embedding
        side_embeds.shape: torch.Size([26495, 64])    多模态融合的embeeding
        '''
        # 多尺度小波兴趣分解
        content_cA, content_cD = self.wavelet_decomp(content_embeds) # 用户与物品信息 # 
        side_cA, side_cD = self.wavelet_decomp(side_embeds) # 多模态辅助信息
        fusion_cA, fusion_cD  = self.wavelet_decomp(content_embeds + side_embeds) # 融合信息
        #print("content_cA.shape:", content_cA.shape, "content_cD.shape:", content_cD.shape, "fusion_cA.shape:", fusion_cA.shape, "fusion_cD.shape:", fusion_cD.shape)
        
        # 低频、高频兴趣融合
        # 低频：
        # low_fused = (content_cA + side_cA + fusion_cA) / 3
        # ------------------------ 低频门控融合 ------------------------
        # 拼接3个低频源特征 [B, 3*(D/2)]
        low_sources = torch.cat([content_cA, side_cA, fusion_cA], dim=-1) #96
        # print("low_sources:", low_sources.shape) 
        # 计算门控权重 [B, 3]
        low_weights = self.low_gate(low_sources)
        # print("low_weights:", low_weights.shape) 
        # 按权重融合（自动广播）
        print("low_weights[:, 1] :", low_weights[:, 1] )
        low_fused = (
            low_weights[:, 0] * content_cA + low_weights[:, 1] * side_cA + low_weights[:, -1] * fusion_cA
        )
        print("low_fused.shape:", low_fused.shape)
        low_fused = self.low_fusion(low_fused)  # 投影层增强表达
        print("low_fused.shape:", low_fused.shape)
        # 高频：
        # high_fused = (content_cD + side_cD + fusion_cD) / 3
        # ------------------------ 高频门控融合 ------------------------
        # 拼接3个高频源特征 [B, 3*(D/2)]
        high_sources = torch.cat([content_cD, side_cD, fusion_cD], dim=-1)
        # 计算门控权重 [B, 3]
        high_weights = self.high_gate(high_sources)
        # 按权重融合（自动广播）
        high_fused = (
            high_weights[:, 0:1] * content_cD +
            high_weights[:, 1:2] * side_cD +
            high_weights[:, 2:-1] * fusion_cD
        )
        high_fused = self.high_fusion(high_fused)  # 投影层增强表达
        #print("high_fused.shape")

        
        # 原始融合兴趣特征
        combined = content_embeds + side_embeds
        # combined = torch.multiply(content_embeds, fusion_embeds)
        # combined = fusion_embeds
        weights = self.attention(torch.concat([low_fused, high_fused], dim=-1))
        #weights = self.attention(combined)
        
        # 加权融合
        reconstructed = self.wavelet_recon(
            weights[:, 0:1] * low_fused,
            weights[:, 1:2] * high_fused
        )
        # reconstructed = self.wavelet_recon(
        #     low_fused,
        #     high_fused
        # )
        low_freq_interest = self.wavelet_recon(low_fused, fusion_cA)
        high_freq_interest = self.wavelet_recon(high_fused, fusion_cD)

        return reconstructed, low_freq_interest, high_freq_interest


In [50]:
# 初始化模型
model = MultiModalWaveletInterestAttention(
    embed_dim=64,
    wavelet_name='db1'
)

# 前向传播（训练模式，带对比学习）
content = torch.randn(512, 64)
side = torch.randn(512, 64)
output, low, high = model(content, side)

# 输出说明：
# - output: 融合后的兴趣特征 [512,64]
# - low: 门控融合后的低频特征（热门兴趣） [512,32]
# - high: 门控融合后的高频特征（小众兴趣） [512,32]
# - loss: 对比学习损失（可用于反向传播）

# 推理模式（无需对比学习）
output, low, high = model(content, side)
print("output:", output, "low:", low, "high:", high)

low_weights[:, 0:1] tensor([[0.5164],
        [0.3512],
        [0.3620],
        [0.2356],
        [0.2120],
        [0.5422],
        [0.6239],
        [0.4749],
        [0.4597],
        [0.4543],
        [0.1630],
        [0.2754],
        [0.3663],
        [0.3108],
        [0.2300],
        [0.2014],
        [0.2407],
        [0.4778],
        [0.5756],
        [0.3742],
        [0.2428],
        [0.5289],
        [0.0832],
        [0.2360],
        [0.4012],
        [0.1926],
        [0.5883],
        [0.1996],
        [0.6465],
        [0.4059],
        [0.2878],
        [0.6077],
        [0.4504],
        [0.8166],
        [0.3190],
        [0.3524],
        [0.2123],
        [0.2737],
        [0.0791],
        [0.5854],
        [0.2888],
        [0.2015],
        [0.2088],
        [0.3559],
        [0.1591],
        [0.2684],
        [0.2341],
        [0.3270],
        [0.1925],
        [0.5931],
        [0.3827],
        [0.1824],
        [0.2978],
        [0.5150],
        

RuntimeError: The size of tensor a (512) must match the size of tensor b (32) at non-singleton dimension 1

In [ ]:
def contrastive_loss(low_feat, high_feat, temperature=0.1):
    """改进的对比损失"""
    # 特征归一化
    low_norm = F.normalize(low_feat, p=2, dim=1)
    high_norm = F.normalize(high_feat, p=2, dim=1)
    
    # 相似度矩阵
    sim_matrix = torch.mm(low_norm, high_norm.T) / temperature
    
    # 对称式损失
    labels = torch.arange(len(low_feat)).to(low_feat.device)
    loss = (F.cross_entropy(sim_matrix, labels) + 
            F.cross_entropy(sim_matrix.T, labels)) / 2
    
    # 正交正则项
    orth_reg = torch.norm(torch.mm(low_norm.T, high_norm)) / len(low_feat)
    
    return loss + 0.1 * orth_reg  # 总损失


In [27]:

def contrastive_loss2( view1, view2, temperature=0.1):
    view1, view2 = F.normalize(view1, dim=1), F.normalize(view2, dim=1)
    pos_score = (view1 * view2).sum(dim=-1)
    pos_score = torch.exp(pos_score / temperature)
    ttl_score = torch.matmul(view1, view2.transpose(0, 1))
    ttl_score = torch.exp(ttl_score / temperature).sum(dim=1)
    cl_loss = -torch.log(pos_score / ttl_score)
    return torch.mean(cl_loss)


In [23]:
contrastive_loss(low, high)

tensor(7.1067)

In [28]:
contrastive_loss2(low, high)

tensor(7.1048)